In [67]:
import re
import sys
import time
import copy
import timeit
import random
import statistics
import numpy as np 
import pandas as pd 
import networkx as nx
import matplotlib.pyplot as plt

from Bio import SeqIO, pairwise2
from Bio.Seq import Seq
from Bio.SeqIO.FastaIO import FastaIterator
from Bio.pairwise2 import format_alignment
from collections import Counter, OrderedDict, namedtuple

import importlib
importlib.reload(sys.modules['RepeatFinder'])

import import_ipynb
from Util import seqInfo, parseFasta, parseSeq, parseSeqByCutter, unifragmentList, fragmentLenPlot
from DataInfo import currDataset, datasetPath, matchPattern, cutter, cutterLen, fragmentN, commonCount
from DataStructure import SeqRepeatInfo, PositionInfo, RepeatEvaInfo, TandemRepeatInfo
from RepeatFinder import findRepeatSeqs, commonRepeatTable, commonRepeatSeqsPosition, commonPositionListToDic, printCommonPositionDic, topRepeatSeqCounter, printTopSeqPercentageTable, integrateRepeatInfo, getIRSComb, evaluateRepeat, generateIRSOutputFile, checkTandemRepeatExist, longestCommonLength, integrateTandemRepeatInfo, generateTROutputFile

importing Jupyter notebook from RepeatFinder.ipynb


In [9]:
parseFastaSeqs = parseFasta(currDataset, datasetPath, matchPattern, matchMode = False)
seqInfo(currDataset, parseFastaSeqs)

...start parsing extractedDanioRerio.fasta fasta file ...
...cost4.437168836593628 sec to parse fasta file ...
extractedDanioRerio.fasta dataset
 number of sequence:5
 total length:332441491



In [10]:
fragmentsLenList, fragmentsSeqList = parseSeqByCutter(parseFastaSeqs)

...start parse seq by cutter: GATC
...cost 4.075203895568848 sec to cut sequence


In [66]:
allRepeatSeqCount, commonRepeatSeq, repeatCount, positionDic = findRepeatSeqs(fragmentsLenList, cutter , fragmentN, commonCount, allRepeatSeqType=1)

... start finding repeat seq ...
...cost1.9143240451812744 sec to finding repeat seq  ...


In [68]:
# tandem Repeat in N fragment
tandemRepeatList = list(filter(checkTandemRepeatExist, allRepeatSeqCount))
tandemRepeatInfoList = integrateTandemRepeatInfo(fragmentsSeqList, fragmentsLenList, tandemRepeatList, positionDic) # integrateTandemRepeatInfo -> only difference is return seq format
generateTROutputFile(tandemRepeatInfoList, matchRatioOfSum=0.4)

In [35]:
# interspersed repetitive sequences
repeatInfoList = integrateRepeatInfo(fragmentsSeqList, fragmentsLenList, allRepeatSeqCount, positionDic)
seqPermutation = getIRSPermutation(repeatInfoList)
generateIRSOutputFile(seqPermutation, matchRatioOfSum=0.4)

In [ ]:
# find repeat seq length list with tolerance
topRepeatSeqKey = commonRepeatInfo[0][0]
topKeySum = sum(list(topRepeatSeqKey))
tolerance = topKeySum *0.05
topRepeatSeqKey

positionKeys = list(positionDic.keys())

matchKeys = []
for i in range(len(positionKeys)):
    temKey = positionKeys[i]
    for j in range(fragmentN):
        flag = 1
        if abs(topRepeatSeqKey[j]- temKey[j]) >= tolerance:
            flag = -1
            break
    if flag == 1:
        matchKeys.append(temKey)
        
matchLenList = []
for i in range(len(matchKeys)):
    matchLenList.append(positionDic[matchKeys[i]])

In [ ]:
flattenMatchSeq = [ j for subset in matchLenList for j in subset ]
candidatePoiDic = dict()
[ candidatePoiDic [t[0]].append(t[1]) if t [0] in list(candidatePoiDic.keys()) else candidatePoiDic.update({t [0]: [t [1]]}) for t in flattenMatchSeq ]
sortedCandidatePoiDic = OrderedDict(sorted(candidatePoiDic.items()))

In [ ]:
matchSeqDic = dict()
for i in sortedCandidatePoiDic:
    chrIndex = i
    matchList = []
    for j in range(len(sortedCandidatePoiDic[chrIndex])):
        startIndex = sortedCandidatePoiDic[chrIndex][j]
        combinedSeq = Seq('').join(fragmentsSeqList[chrIndex][startIndex:startIndex+fragmentN])
        matchList.append(combinedSeq)
    matchSeqDic[chrIndex] = matchList

In [ ]:
# # commonCount repeat sequences
# commonPositionList = commonRepeatSeqsPosition(fragmentsLenList, commonRepeatInfo, positionDic, commonCount)
# commonPositionDic = commonPositionListToDic(commonPositionList)

# # the top common repeat sequence
# singleRepeatSeqCount, comRepeatSeqCount = topRepeatSeqCounter(fragmentsSeqList, commonRepeatInfo, positionDic, fragmentN)printTopSeqPercentageTable(comRepeatSeqCount, showPartialSeq=True)